In [1]:
%%bash
set -e

python3 -m pip install -U virtualenv
rm -rf /content/ads_venv
python3 -m virtualenv /content/ads_venv

/content/ads_venv/bin/python -m pip install -U pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 27.3 MB/s eta 0:00:00
created virtual environment CPython3.12.12.final.0-64 in 323ms
  creator CPython3Posix(dest=/content/ads_venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s  0:00:00



In [2]:
%%bash
set -e

/content/ads_venv/bin/python -m pip install -U \
  streamlit \
  langchain langchain-core langchain-google-genai google-generativeai \
  "e2b-code-interpreter==2.4.1" \
  pandas matplotlib seaborn \
  "pillow<12"

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 32.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 78.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 126.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 46.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 181.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 113.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 49.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.4/719.4 kB 42.7 MB/s  0:00:00
   ━━━━━

In [3]:
import os
os.makedirs("/content/.streamlit", exist_ok=True)

with open("/content/.streamlit/secrets.toml", "w") as f:
    f.write(
        'GOOGLE_API_KEY="PASTE_YOUR_API_KEY"\n'
        'E2B_API_KEY="PASTE_YOUR_API_KEY"\n'
    )

print("Saved keys to /content/.streamlit/secrets.toml")

Saved keys to /content/.streamlit/secrets.toml


In [4]:
!/content/ads_venv/bin/python -m pip install -U google-generativeai langchain-google-genai langchain langchain-core

In [5]:
%%bash
set -e
/content/ads_venv/bin/python - <<'PY'
import tomllib
import google.generativeai as genai

secrets = tomllib.load(open("/content/.streamlit/secrets.toml","rb"))
genai.configure(api_key=secrets["GOOGLE_API_KEY"])

print("Models supporting generateContent:\n")
for m in genai.list_models():
    if "generateContent" in getattr(m, "supported_generation_methods", []):
        print(m.name)
PY

Models supporting generateContent:

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


<stdin>:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md



In [6]:
# app.py
# End-to-end patched for:
# - Gemini models: supports models/* ids + fallback; normalizes list content -> string
# - E2B e2b-code-interpreter==2.4.1: MUST use Sandbox.create()
# - Colab-friendly: run via /content/ads_venv/bin/python -m streamlit run app.py
%%writefile /content/app.py
import os
import re
import io
import base64
import tempfile
import traceback
from typing import Any, Dict, List, Optional, Tuple

import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# E2B (your working path)
try:
    from e2b_code_interpreter.code_interpreter_sync import Sandbox as E2BSandbox  # type: ignore
    E2B_BACKEND = "e2b_code_interpreter.code_interpreter_sync.Sandbox"
except Exception:
    from e2b_code_interpreter import Sandbox as E2BSandbox  # type: ignore
    E2B_BACKEND = "e2b_code_interpreter.Sandbox"


# ----------------------------
# Page config
# ----------------------------
st.set_page_config(page_title="Autonomous Data Science Agent", layout="wide")


# ----------------------------
# Secrets / Config
# ----------------------------
def get_secret(name: str) -> Optional[str]:
    try:
        v = st.secrets.get(name)  # type: ignore[attr-defined]
        if v:
            return str(v)
    except Exception:
        pass
    return os.getenv(name)


if "GOOGLE_API_KEY" not in st.session_state:
    st.session_state.GOOGLE_API_KEY = get_secret("GOOGLE_API_KEY")

if "E2B_API_KEY" not in st.session_state:
    st.session_state.E2B_API_KEY = get_secret("E2B_API_KEY")

# Models you confirmed are available for generateContent
SUPPORTED_MODELS = [
    "models/gemini-flash-latest",
    "models/gemini-pro-latest",
    "models/gemini-2.5-flash",
    "models/gemini-2.5-pro",
    "models/gemini-2.0-flash",
]

if "GEMINI_MODEL" not in st.session_state:
    st.session_state.GEMINI_MODEL = "models/gemini-flash-latest"


# ----------------------------
# Helpers
# ----------------------------
CODE_BLOCK_RE = re.compile(r"```(?:python)?\s*(.*?)```", re.DOTALL | re.IGNORECASE)


def llm_content_to_text(content: Any) -> str:
    """
    Gemini/LangChain sometimes returns content as a LIST of parts.
    Convert to a single string safe for regex parsing.
    """
    if content is None:
        return ""
    if isinstance(content, str):
        return content
    if isinstance(content, list):
        parts: List[str] = []
        for p in content:
            if p is None:
                continue
            if isinstance(p, str):
                parts.append(p)
            elif isinstance(p, dict):
                # common shapes: {"text": "..."} or {"type":"text","text":"..."}
                if "text" in p and isinstance(p["text"], str):
                    parts.append(p["text"])
                else:
                    parts.append(str(p))
            else:
                # objects with .text, etc.
                t = getattr(p, "text", None)
                if isinstance(t, str):
                    parts.append(t)
                else:
                    parts.append(str(p))
        return "".join(parts)
    return str(content)


def extract_code_and_text(model_output: str) -> Tuple[str, str]:
    matches = CODE_BLOCK_RE.findall(model_output or "")
    code = matches[0].strip() if matches else ""
    text = CODE_BLOCK_RE.sub("", model_output or "").strip()
    if not text and code:
        text = "Executed the analysis in the sandbox. See outputs below."
    return code, text


def safe_filename(name: str) -> str:
    name = name or "dataset.csv"
    return re.sub(r"[^A-Za-z0-9_.-]", "_", name)


def build_system_prompt(remote_path: str) -> str:
    return f"""
You are an Expert Python Data Scientist. You have access to a dataset at path: {remote_path}

Rules:
- Be concise. Avoid conversational filler (e.g., "Here is the code").
- Always read the dataset using pandas: df = pd.read_csv(r"{remote_path}")
- Use pandas, matplotlib, seaborn as needed.
- If a chart is relevant, generate it and ALWAYS call plt.show() so it is captured.
- Return your response in TWO parts:
  1) A short answer (bullets are fine).
  2) ONE executable Python code block fenced as ```python ... ``` (no extra code blocks).
- The code must be runnable as-is in a fresh Python environment.
""".strip()


def normalize_logs(v: Any) -> str:
    """E2B 2.4.1 logs are lists; normalize to string."""
    if v is None:
        return ""
    if isinstance(v, str):
        return v
    if isinstance(v, list):
        try:
            return "".join(map(str, v))
        except Exception:
            return "\n".join([repr(x) for x in v])
    return str(v)


def _b64_to_bytes(s: str) -> Optional[bytes]:
    try:
        return base64.b64decode(s)
    except Exception:
        return None


def coerce_image_to_bytes(obj: Any) -> Optional[bytes]:
    try:
        if obj is None:
            return None

        if isinstance(obj, (bytes, bytearray)):
            return bytes(obj)

        if isinstance(obj, str):
            return _b64_to_bytes(obj)

        if isinstance(obj, dict):
            for k in ("png", "data", "base64"):
                v = obj.get(k)
                if isinstance(v, (bytes, bytearray)):
                    return bytes(v)
                if isinstance(v, str):
                    b = _b64_to_bytes(v)
                    if b:
                        return b

        for attr in ("png", "data", "base64"):
            if hasattr(obj, attr):
                v = getattr(obj, attr)
                if isinstance(v, (bytes, bytearray)):
                    return bytes(v)
                if isinstance(v, str):
                    b = _b64_to_bytes(v)
                    if b:
                        return b

        # PIL fallback
        try:
            from PIL import Image  # type: ignore

            if isinstance(obj, Image.Image):
                buf = io.BytesIO()
                obj.save(buf, format="PNG")
                return buf.getvalue()
        except Exception:
            pass

        return None
    except Exception:
        return None


# ----------------------------
# E2B Sandbox Manager (E2B 2.4.1: Sandbox.create() only)
# ----------------------------
class SandboxHandler:
    def __init__(self, e2b_api_key: str):
        if not e2b_api_key:
            raise ValueError("E2B_API_KEY is missing.")

        os.environ["E2B_API_KEY"] = e2b_api_key

        if not (hasattr(E2BSandbox, "create") and callable(getattr(E2BSandbox, "create"))):
            raise RuntimeError(f"{E2B_BACKEND} does not expose Sandbox.create().")

        self.sandbox = E2BSandbox.create()
        self.init_mode = "Sandbox.create()"
        self.backend = E2B_BACKEND

        try:
            self.sandbox.run_code(
                "import os\n"
                "os.makedirs('/home/user/data', exist_ok=True)\n"
                "print('Sandbox ready')\n"
            )
        except Exception:
            pass

    def upload_file(self, uploaded_file) -> str:
        if uploaded_file is None:
            raise ValueError("No file provided.")
        data = uploaded_file.getvalue()
        if not data:
            raise ValueError("Uploaded file is empty.")

        fname = safe_filename(getattr(uploaded_file, "name", "dataset.csv"))
        remote_path = f"/home/user/data/{fname}"

        # E2B 2.4.1 supports files.write(path, bytes)
        self.sandbox.files.write(remote_path, data)  # type: ignore[attr-defined]
        return remote_path

    def run_code(self, code: str) -> Dict[str, Any]:
        if not code or not code.strip():
            return {"stdout": "", "stderr": "No code to execute.", "images": [], "raw": None}

        try:
            execution = self.sandbox.run_code(code)
        except Exception as e:
            return {
                "stdout": "",
                "stderr": f"Sandbox execution failed: {e}\n\n{traceback.format_exc()}",
                "images": [],
                "raw": None,
            }

        stdout, stderr = "", ""
        images: List[bytes] = []

        try:
            logs = getattr(execution, "logs", None)
            if logs is not None:
                stdout = normalize_logs(getattr(logs, "stdout", ""))
                stderr = normalize_logs(getattr(logs, "stderr", ""))
            else:
                stdout = normalize_logs(getattr(execution, "stdout", ""))
                stderr = normalize_logs(getattr(execution, "stderr", ""))
        except Exception:
            pass

        try:
            for img in (getattr(execution, "images", None) or []):
                b = coerce_image_to_bytes(img)
                if b:
                    images.append(b)
        except Exception:
            pass

        try:
            for r in (getattr(execution, "results", None) or []):
                b = coerce_image_to_bytes(r)
                if b:
                    images.append(b)
        except Exception:
            pass

        return {"stdout": stdout, "stderr": stderr, "images": images, "raw": execution}


# ----------------------------
# Cached resources
# ----------------------------
@st.cache_resource(show_spinner=False)
def get_sandbox_handler(e2b_key: str) -> SandboxHandler:
    return SandboxHandler(e2b_key)


@st.cache_resource(show_spinner=False)
def get_llm(google_key: str, model_name: str) -> ChatGoogleGenerativeAI:
    return ChatGoogleGenerativeAI(
        model=model_name,
        temperature=0.2,
        google_api_key=google_key,
    )


def invoke_with_model_fallback(
    google_key: str,
    model_name: str,
    messages: List[Any],
    fallback_model: str = "models/gemini-flash-latest",
) -> str:
    """
    Always returns TEXT (string). Handles Gemini content being list-of-parts.
    Also retries on model 404/NOT_FOUND using fallback model.
    """
    try:
        llm = get_llm(google_key, model_name)
        resp = llm.invoke(messages)
        return llm_content_to_text(getattr(resp, "content", resp))
    except Exception as e:
        msg = str(e)
        if ("404" in msg) or ("NOT_FOUND" in msg) or ("not found" in msg.lower()):
            llm_fb = get_llm(google_key, fallback_model)
            resp2 = llm_fb.invoke(messages)
            return llm_content_to_text(getattr(resp2, "content", resp2))
        raise


# ----------------------------
# Session state init
# ----------------------------
if "messages" not in st.session_state:
    st.session_state.messages = []
if "remote_path" not in st.session_state:
    st.session_state.remote_path = None
if "uploaded_filename" not in st.session_state:
    st.session_state.uploaded_filename = None


# ----------------------------
# Sidebar UI
# ----------------------------
with st.sidebar:
    st.header("Configuration")

    st.text_input("GOOGLE_API_KEY", type="password",
                  value=st.session_state.GOOGLE_API_KEY or "", key="GOOGLE_API_KEY")
    st.text_input("E2B_API_KEY", type="password",
                  value=st.session_state.E2B_API_KEY or "", key="E2B_API_KEY")

    st.selectbox(
        "Gemini model",
        options=SUPPORTED_MODELS,
        index=SUPPORTED_MODELS.index(st.session_state.GEMINI_MODEL)
        if st.session_state.GEMINI_MODEL in SUPPORTED_MODELS else 0,
        key="GEMINI_MODEL",
    )

    if not st.session_state.GOOGLE_API_KEY:
        st.warning("Missing GOOGLE_API_KEY (st.secrets or env).")
    else:
        st.success("GOOGLE_API_KEY loaded.")

    if not st.session_state.E2B_API_KEY:
        st.warning("Missing E2B_API_KEY (st.secrets or env).")
    else:
        st.success("E2B_API_KEY loaded.")

    st.caption(f"E2B backend: `{E2B_BACKEND}`")

    if st.button("Reset sandbox/cache"):
        try:
            st.cache_resource.clear()
        except Exception:
            pass
        st.session_state.remote_path = None
        st.session_state.uploaded_filename = None
        st.session_state.messages = []
        st.rerun()

    st.divider()
    st.subheader("Dataset Upload")

    uploaded = st.file_uploader("Upload a CSV file", type=["csv"])
    if uploaded is not None:
        try:
            if not st.session_state.E2B_API_KEY:
                raise RuntimeError("E2B_API_KEY is missing.")
            sb = get_sandbox_handler(st.session_state.E2B_API_KEY)
            st.caption(f"E2B init: `{sb.init_mode}`")

            if st.session_state.uploaded_filename != uploaded.name or st.session_state.remote_path is None:
                remote_path = sb.upload_file(uploaded)
                st.session_state.remote_path = remote_path
                st.session_state.uploaded_filename = uploaded.name
                st.success("Dataset Uploaded Successfully")
                st.caption(f"Sandbox path: `{remote_path}`")
                st.session_state.messages = []
        except Exception as e:
            st.error(f"Upload failed: {e}")


# ----------------------------
# Main UI
# ----------------------------
st.title("🤖 Autonomous Data Science Agent")

if not st.session_state.remote_path:
    st.info("Upload a CSV in the sidebar to begin.")
    st.stop()

if not st.session_state.GOOGLE_API_KEY:
    st.error("GOOGLE_API_KEY is required.")
    st.stop()

if not st.session_state.E2B_API_KEY:
    st.error("E2B_API_KEY is required.")
    st.stop()

# Render chat history
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

user_query = st.chat_input("Ask a question about your dataset (EDA, plots, modeling, etc.)")

if user_query:
    st.session_state.messages.append({"role": "user", "content": user_query})
    with st.chat_message("user"):
        st.markdown(user_query)

    remote_path = st.session_state.remote_path
    system_prompt = build_system_prompt(remote_path)

    chosen_model = st.session_state.GEMINI_MODEL
    fallback_model = "models/gemini-flash-latest"

    sb = get_sandbox_handler(st.session_state.E2B_API_KEY)

    lc_messages: List[Any] = [SystemMessage(content=system_prompt)]
    for m in st.session_state.messages:
        lc_messages.append(
            HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"])
        )

    with st.chat_message("assistant"):
        with st.spinner("🤖 Analyzing data..."):
            # 1) Gemini -> (text+code)
            try:
                model_output = invoke_with_model_fallback(
                    st.session_state.GOOGLE_API_KEY,
                    chosen_model,
                    lc_messages,
                    fallback_model=fallback_model,
                )
            except Exception as e:
                st.error(f"Gemini request failed: {e}")
                st.stop()

            code, assistant_text = extract_code_and_text(model_output)

            # Force code if missing
            if not code.strip():
                force = (
                    "Return ONLY one executable Python code block fenced as ```python ... ```.\n"
                    f"Dataset path: {remote_path}\n"
                    f"Must include: df = pd.read_csv(r\"{remote_path}\")\n"
                    "If you generate plots, call plt.show()."
                )
                model_output = invoke_with_model_fallback(
                    st.session_state.GOOGLE_API_KEY,
                    chosen_model,
                    lc_messages + [HumanMessage(content=force)],
                    fallback_model=fallback_model,
                )
                code, assistant_text2 = extract_code_and_text(model_output)
                if assistant_text2:
                    assistant_text = assistant_text2

            # 2) Execute + self-correct loop
            final_text = assistant_text
            final_code = code
            final_stdout = ""
            final_stderr = ""
            final_images: List[bytes] = []

            retries = 0
            last_exec: Optional[Dict[str, Any]] = None

            while retries < 3:
                last_exec = sb.run_code(final_code)
                final_stdout = last_exec.get("stdout", "") or ""
                final_stderr = last_exec.get("stderr", "") or ""
                final_images = last_exec.get("images", []) or []

                if final_stderr.strip():
                    retries += 1
                    fix_prompt = (
                        "You got this error. Fix the code and try again.\n\n"
                        f"{final_stderr}\n\n"
                        "Return ONLY one executable Python code block fenced as ```python ... ```.\n"
                        f"Dataset path: {remote_path}\n"
                        f"Must include: df = pd.read_csv(r\"{remote_path}\")\n"
                        "If you generate plots, call plt.show()."
                    )
                    fix_out = invoke_with_model_fallback(
                        st.session_state.GOOGLE_API_KEY,
                        chosen_model,
                        [SystemMessage(content=system_prompt), *lc_messages[1:], AIMessage(content=model_output), HumanMessage(content=fix_prompt)],
                        fallback_model=fallback_model,
                    )
                    code2, _ = extract_code_and_text(fix_out)
                    if code2.strip():
                        final_code = code2
                        model_output = fix_out
                        continue
                    break
                else:
                    break

        st.markdown(final_text)

        if final_images:
            for i, b in enumerate(final_images, start=1):
                st.image(b, caption=f"Chart {i}", use_container_width=True)

        with st.expander("See executed code"):
            st.code(final_code or "# (no code)", language="python")
            if final_stdout.strip():
                st.subheader("stdout")
                st.code(final_stdout, language="text")
            if final_stderr.strip():
                st.subheader("stderr")
                st.code(final_stderr, language="text")

    st.session_state.messages.append({"role": "assistant", "content": final_text})

Writing /content/app.py


In [7]:
%%bash
set -e

pkill -f "streamlit run /content/app.py" || true

nohup /content/ads_venv/bin/python -m streamlit run /content/app.py \
  --server.address 0.0.0.0 --server.port 8501 \
  --server.enableCORS false --server.enableXsrfProtection false \
  --browser.gatherUsageStats false \
  > /content/streamlit.log 2>&1 &

In [8]:
!tail -n 80 /content/streamlit.log
!curl -I http://localhost:8501 | head


  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  1522    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
HTTP/1.1 200 OK
Server: TornadoServer/6.5.4
Content-Type: text/html
Date: Thu, 29 Jan 2026 15:48:54 GMT
Accept-Ranges: bytes
Etag: "7d9e795beb02d5dcdf74cc38d352342ed3b6d5c2b91e29e46d25d795c1aade0dc6df0aa9c6141ebf9d712fd99d3cb1aaf9ac2b3dc74acd445e3c631dbfcc847b"
Last-Modified: Thu, 29 Jan 2026 15:48:01 GMT
Cache-Control: no-cache
Content-Length: 1522
Vary: Accept-Encoding


In [15]:
import socket, subprocess, time, re

def get_free_port():
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

PORT = get_free_port()
print("Using port:", PORT)

# Kill previous runs (safe)
subprocess.run("pkill -f 'streamlit run /content/app.py' || true", shell=True)
subprocess.run("pkill -f 'cloudflared tunnel --url' || true", shell=True)

# Start Streamlit in background
subprocess.Popen(
    ["/content/ads_venv/bin/python", "-m", "streamlit", "run", "/content/app.py",
     "--server.address", "0.0.0.0",
     "--server.port", str(PORT),
     "--server.enableCORS", "false",
     "--server.enableXsrfProtection", "false"],
    stdout=open("/content/streamlit.log", "w"),
    stderr=subprocess.STDOUT
)

# Wait until Streamlit is reachable
for _ in range(30):
    r = subprocess.run(["bash", "-lc", f"curl -s -o /dev/null -w '%{{http_code}}' http://localhost:{PORT}"],
                       capture_output=True, text=True)
    if r.stdout.strip() in ("200", "302"):
        print("Streamlit is up.")
        break
    time.sleep(1)
else:
    print("Streamlit did not start. Check /content/streamlit.log")

Using port: 42185
Streamlit is up.


In [16]:
# Replace 12345 with the printed port
!/usr/local/bin/cloudflared tunnel --url http://localhost:42185 --no-autoupdate --loglevel info

2026-01-29T15:51:48Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-29T15:51:48Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-29T15:51:52Z INF +--------------------------------------------------------------------------------------------+
2026-01-29T15:51:52Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-29T15:51:52Z INF |  https://performer-deferred-rec-belfast.trycloudflare.